# Agent with Web Search Tool
https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/web-search?view=foundry&pivots=python

In [4]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, WebSearchPreviewTool, ApproximateLocation

load_dotenv()


endpoint = os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"]

In [5]:
with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=endpoint, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
):
    # [START tool_declaration]
    tool = WebSearchPreviewTool(user_location=ApproximateLocation(country="GB", city="London", region="London"))
    # [END tool_declaration]

    # Create Agent with web search tool
    agent = project_client.agents.create_version(
        agent_name="MyAgent",
        definition=PromptAgentDefinition(
            model=os.environ["AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME"],
            instructions="You are a helpful assistant that can search the web",
            tools=[tool],
        ),
        description="Agent for web search.",
    )
    print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

    # Create a conversation for the agent interaction
    conversation = openai_client.conversations.create()
    print(f"Created conversation (id: {conversation.id})")

    # Send a query to search the web
    response = openai_client.responses.create(
        conversation=conversation.id,
        input="Show me the latest London Underground service updates",
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )
    print(f"==> Result: {response.output_text}")

    print("\nCleaning up...")
    project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
    print("Agent deleted")


Agent created (id: MyAgent:5, name: MyAgent, version: 5)
Created conversation (id: conv_0c2ffe8f0912e82400vaDFIZuTisvcou2HYK0tD3AHKrxzuAwY)
==> Result: Here are the *live* London Underground updates from Transport for London right now (showing **Tube lines only**):

- **Bakerloo** – Good service  
- **Central** – Good service  
- **Circle** – Good service  
- **District** – Minor delays between *Earl’s Court and Ealing Broadway/Richmond* and *Barking to Upminster eastbound only* due to an earlier points failure. Good service on the rest of the line.   
- **Hammersmith & City** – Good service  
- **Jubilee** – Good service  
- **Metropolitan** – Minor delays between *Moor Park and Watford* due to train cancellations. Good service on the rest of the line.   
- **Northern** – Good service  
- **Piccadilly** – Minor delays between *Acton Town and Uxbridge* due to a points failure at Acton Town. Good service on the rest of the line.   
- **Victoria** – Good service  
- **Waterloo & City** –